In [63]:
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import os
import random
from random import shuffle
import scipy.ndimage
import tensorflow as tf

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.activations import softmax
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv3D, Reshape, BatchNormalization, MaxPooling3D, Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD, Adam, Nadam, Adadelta, Adagrad, Adamax
from tensorflow.keras import backend as K
#K.set_image_dim_ordering('th')
import np_utils
from tensorflow.keras.utils import plot_model
#from keract import get_activations, display_activations
from tensorflow import one_hot as to_categorical


from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io as sci
import time
from sklearn import random_projection as rp
#from skimage.filters import gabor, gaussian
import scipy
import spectral
import time
import math
import pydot
import graphviz

In [64]:
windowSize = 11
testRatio = 0.9
valRatio = 0.1

In [65]:
def loadIndianPinesData():
    data = sio.loadmat(r'C:/Users/bishw/OneDrive/Desktop/Indian_pines_corrected.mat')['indian_pines_corrected']
    labels = sio.loadmat(r'C:/Users/bishw/OneDrive/Desktop/Indian_pines_gt.mat')['indian_pines_gt']
    return data, labels

In [66]:
X, y = loadIndianPinesData()
print ("Shape of X : "+str(X.shape))
print ("Shape of y : "+str(y.shape))

#For plotting accuracy plots
y_full = y

Shape of X : (145, 145, 200)
Shape of y : (145, 145)


In [67]:
def standartizeData(X):
    newX = np.reshape(X, (-1, X.shape[2]))
    scaler = preprocessing.StandardScaler()  
    newX = scaler.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1],X.shape[2]))
    return newX, scaler

In [68]:
X, scaler = standartizeData(X)
print ("Shape of X : "+str(X.shape))

C:\Users\bishw\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\bishw\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Shape of X : (145, 145, 200)


In [69]:
def applyPCA(X, numComponents=50):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [70]:
X, pca = applyPCA(X)

print ("Shape of X : "+str(X.shape))

#For plotting accuracy plots
X_full = X

Shape of X : (145, 145, 50)


In [71]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createPatches(X, y, windowSize=7, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [72]:
X, y = createPatches(X, y, windowSize=windowSize)
print ("Shape of X : "+str(X.shape))
print ("Shape of y : "+str(y.shape))

Shape of X : (10249, 11, 11, 50)
Shape of y : (10249,)


In [73]:
def splitTrainTestSet(X, y, testRatio=0.9):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=345, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=testRatio/(testRatio + valRatio), random_state=345, stratify=y_test)
    return X_train, X_test, X_val, y_train, y_test, y_val

In [74]:
X_train, X_test,X_val, y_train, y_test, y_val = splitTrainTestSet(X, y, testRatio)
print ("Shape of X_train : "+str(X_train.shape))
print ("Shape of X_test : "+str(X_test.shape))
print ("Shape of X_val : "+str(X_val.shape))
print ("Shape of y_train : "+str(y_train.shape))
print ("Shape of y_test : "+str(y_test.shape))
print ("Shape of y_val : "+str(y_val.shape))

Shape of X_train : (1024, 11, 11, 50)
Shape of X_test : (8303, 11, 11, 50)
Shape of X_val : (922, 11, 11, 50)
Shape of y_train : (1024,)
Shape of y_test : (8303,)
Shape of y_val : (922,)


In [75]:
count = y_train
import collections
counter=collections.Counter(count)
print (counter)

Counter({10.0: 245, 1.0: 143, 13.0: 126, 9.0: 97, 2.0: 83, 5.0: 73, 11.0: 59, 4.0: 48, 7.0: 48, 14.0: 39, 3.0: 24, 12.0: 20, 15.0: 9, 0.0: 5, 6.0: 3, 8.0: 2})


In [76]:
def oversampleWeakClasses(X, y):
    uniqueLabels, labelCounts = np.unique(y, return_counts=True)
    maxCount = np.max(labelCounts)
    labelInverseRatios = maxCount / labelCounts  
    # repeat for every label and concat
    newX = X[y == uniqueLabels[0], :, :, :].repeat(round(labelInverseRatios[0]), axis=0)
    newY = y[y == uniqueLabels[0]].repeat(round(labelInverseRatios[0]), axis=0)
    for label, labelInverseRatio in zip(uniqueLabels[1:], labelInverseRatios[1:]):
        cX = X[y== label,:,:,:].repeat(round(labelInverseRatio), axis=0)
        cY = y[y == label].repeat(round(labelInverseRatio), axis=0)
        newX = np.concatenate((newX, cX))
        newY = np.concatenate((newY, cY))
    np.random.seed(seed=42)
    rand_perm = np.random.permutation(newY.shape[0])
    newX = newX[rand_perm, :, :, :]
    newY = newY[rand_perm]
    return newX, newY

In [77]:
X_train,  y_train = oversampleWeakClasses(X_train, y_train)
print ("Shape of X_train : "+str(X_train.shape))
print ("Shape of X_test : "+str(X_test.shape))
print ("Shape of X_val : "+str(X_val.shape))
print ("Shape of y_train : "+str(y_train.shape))
print ("Shape of y_test : "+str(y_test.shape))
print ("Shape of y_val : "+str(y_val.shape))

Shape of X_train : (3950, 11, 11, 50)
Shape of X_test : (8303, 11, 11, 50)
Shape of X_val : (922, 11, 11, 50)
Shape of y_train : (3950,)
Shape of y_test : (8303,)
Shape of y_val : (922,)


In [78]:
count = y_train
import collections
counter=collections.Counter(count)
print (counter)

Counter({9.0: 291, 1.0: 286, 13.0: 252, 2.0: 249, 6.0: 246, 0.0: 245, 10.0: 245, 8.0: 244, 15.0: 243, 3.0: 240, 4.0: 240, 7.0: 240, 12.0: 240, 11.0: 236, 14.0: 234, 5.0: 219})


In [79]:
def AugmentData(X_train):
    for i in range(int(X_train.shape[0]/2)):
        patch = X_train[i,:,:,:]
        num = random.randint(0,2)
        if (num == 0):
            
            flipped_patch = np.flipud(patch)
        if (num == 1):
            
            flipped_patch = np.fliplr(patch)
        if (num == 2):
            
            no = random.randrange(-180,180,30)
            flipped_patch = scipy.ndimage.interpolation.rotate(patch, no,axes=(1, 0),
                                                               reshape=False, output=None, order=3, mode='constant', cval=0.0, prefilter=False)
    
    
    patch2 = flipped_patch
    X_train[i,:,:,:] = patch2
    
    return X_train

In [80]:
X_train = AugmentData(X_train)
print ("Shape of X_train : "+str(X_train.shape))
print ("Shape of X_test : "+str(X_test.shape))
print ("Shape of X_val : "+str(X_val.shape))
print ("Shape of y_train : "+str(y_train.shape))
print ("Shape of y_test : "+str(y_test.shape))
print ("Shape of y_val : "+str(y_val.shape))

Shape of X_train : (3950, 11, 11, 50)
Shape of X_test : (8303, 11, 11, 50)
Shape of X_val : (922, 11, 11, 50)
Shape of y_train : (3950,)
Shape of y_test : (8303,)
Shape of y_val : (922,)


In [81]:
X_train_RNN = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3], 1)
X_test_RNN = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3], 1)
X_val_RNN = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], X_val.shape[3], 1)

print ("Shape of X_train_RNN : "+str(X_train_RNN.shape))
print ("Shape of X_test_RNN : "+str(X_test_RNN.shape))
print ("Shape of X_val_RNN : "+str(X_val_RNN.shape))

Shape of X_train_RNN : (3950, 11, 11, 50, 1)
Shape of X_test_RNN : (8303, 11, 11, 50, 1)
Shape of X_val_RNN : (922, 11, 11, 50, 1)


In [82]:
X_train_RES = X_train
X_test_RES = X_test
X_val_RES = X_val

print ("Shape of X_train_RES : "+str(X_train_RES.shape))
print ("Shape of X_test_RES : "+str(X_test_RES.shape))
print ("Shape of X_val_RES : "+str(X_val_RES.shape))

Shape of X_train_RES : (3950, 11, 11, 50)
Shape of X_test_RES : (8303, 11, 11, 50)
Shape of X_val_RES : (922, 11, 11, 50)


In [83]:
y_train = tf.cast(y_train, tf.int32)
y_test = tf.cast(y_test, tf.int32)
y_val = tf.cast(y_val, tf.int32)

y_train = to_categorical(indices = y_train, depth = 16)
y_test = to_categorical(indices = y_test, depth = 16)
y_val = to_categorical(indices = y_val, depth = 16)

y_train = y_train.numpy()
y_test = y_test.numpy()
y_val = y_val.numpy()

print ("Shape of X_train_RNN : "+str(X_train_RNN.shape))
print ("Shape of X_test_RNN : "+str(X_test_RNN.shape))
print ("Shape of X_val_RNN : "+str(X_val_RNN.shape))

print ("Shape of X_train_RES : "+str(X_train_RES.shape))
print ("Shape of X_test_RES : "+str(X_test_RES.shape))
print ("Shape of X_val_RES : "+str(X_val_RES.shape))

print ("Shape of y_train : "+str(y_train.shape))
print ("Shape of y_test : "+str(y_test.shape))
print ("Shape of y_val : "+str(y_val.shape))

Shape of X_train_RNN : (3950, 11, 11, 50, 1)
Shape of X_test_RNN : (8303, 11, 11, 50, 1)
Shape of X_val_RNN : (922, 11, 11, 50, 1)
Shape of X_train_RES : (3950, 11, 11, 50)
Shape of X_test_RES : (8303, 11, 11, 50)
Shape of X_val_RES : (922, 11, 11, 50)
Shape of y_train : (3950, 16)
Shape of y_test : (8303, 16)
Shape of y_val : (922, 16)


In [84]:
input_RNN_shape = X_train_RNN[0].shape
input_RES_shape = X_train_RES[0].shape

inputRNN = Input(shape= input_RNN_shape)
x = Conv3D(filters=16, kernel_size=(3,3,32), data_format='channels_last')(inputRNN)
x = AveragePooling3D(pool_size=(2,2,2), data_format='channels_last')(x)
x = Conv3D(filters=32, kernel_size=(3,3,8), data_format='channels_last')(x)
x = Flatten()(x)
x = Reshape((x.shape[1],1))(x)
y = x
x = Bidirectional(LSTM(1, return_sequences = True), merge_mode='mul')(x)
x = Bidirectional(LSTM(1, return_sequences = True), merge_mode='mul')(x)
x = Flatten()(x)
x = Activation(activation='softmax')(x)
x = Reshape((x.shape[1],1))(x)
x = Multiply()([y, x])
x = Add()([y, x])
x = Flatten()(x)
x = Dense(units = 100, activation='relu', kernel_regularizer='l2')(x)
x = Dropout(rate = 0.2)(x)
x = Dense(units = 50, activation='relu', kernel_regularizer='l2')(x)
#x = Dense(units= 16, activation='softmax')(x)
modelRNN = Model(inputs=inputRNN, outputs=x)

#######################################################################################################################

inputRES = Input(shape= input_RES_shape)
y = inputRES
y = Reshape((windowSize,windowSize,50,1))(y)
y = Conv3D(filters=1, kernel_size=(1,1,25),activation='relu', data_format='channels_last')(y)
y = Conv3D(filters=1, kernel_size=(1,1,26),activation='relu', data_format='channels_last')(y)
y = Flatten()(y)
y = Activation(activation='softmax')(y)
y = Reshape((windowSize,windowSize,1))(y)
#y = tf.keras.backend.repeat_elements(y, X_test_RES.shape[3], axis = 3)
#y = repeat(y, repeats=X_test_RES.shape[3], axis = -1 )
y = tf.tile(input=y, multiples=[1,1,1,X_test_RES.shape[3]])
y = multiply([y, inputRES])
y = Add()([y, inputRES])

#y = Flatten()(y)
#y = Dense(units=16, activation='softmax')(y)

#First layer of ResNet
skip1 = y
skip1 = Conv2D(filters = 32, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last')(skip1)
skip1 = BatchNormalization()(skip1)
skip1 = Activation(activation='relu')(skip1)
skip1 = Conv2D(filters = 64, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last')(skip1)
skip1 = BatchNormalization()(skip1)
y = Conv2D(filters = 64, kernel_size=(1,1), strides=(1,1), padding='same', data_format='channels_last')(y)
y = Add()([y, skip1])
y = Activation(activation='relu')(y)
y = AveragePooling2D(pool_size=(2,2), data_format='channels_last')(y)

#Second layer of ResNet
skip2 = y
skip2 = Conv2D(filters = 32, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last')(skip2)
skip2 = BatchNormalization()(skip2)
skip2 = Activation(activation='relu')(skip2)
skip2 = Conv2D(filters = 64, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last')(skip2)
skip2 = BatchNormalization()(skip2)
y = Conv2D(filters = 64, kernel_size=(1,1), strides=(1,1), padding='same', data_format='channels_last')(y)
y = Add()([y, skip2])
y = Activation(activation='relu')(y)
#extra addition
#y = AveragePooling2D(pool_size=(2,2), data_format='channels_last')(y)

#Third layer of ResNet
skip3 = y
skip3 = Conv2D(filters = 32, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last')(skip3)
skip3 = BatchNormalization()(skip3)
skip3 = Activation(activation='relu')(skip3)
skip3 = Conv2D(filters = 64, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last')(skip3)
skip3 = BatchNormalization()(skip3)
y = Conv2D(filters = 64, kernel_size=(1,1), strides=(1,1), padding='same', data_format='channels_last')(y)
y = Add()([y, skip3])
y = Activation(activation='relu')(y)
y = AveragePooling2D(pool_size=(2,2), data_format='channels_last')(y)

#Dense Layers
y = Flatten()(y)
y = Dense(units = 100, activation='relu', kernel_regularizer='l2')(y)
y = Dropout(rate = 0.2)(y)
y = Dense(units= 50, activation='relu', kernel_regularizer='l2')(y)
#y = Dense(units= 16, activation='softmax')(y)

modelRES = Model(inputs=inputRES, outputs=y)

#############################################################################################################################

#Bringing the output of both the models together to final dense layers
z = add([modelRNN.output, modelRES.output])
z = Dense(units = 25, activation='relu', kernel_regularizer='l2')(z)
z = Dropout(rate = 0.2)(z)
z = Dense(units= y_train.shape[1], activation='softmax')(z)


model = Model(inputs=[modelRNN.input, modelRES.input], outputs=z)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 11, 11, 50)] 0                                            
__________________________________________________________________________________________________
reshape_10 (Reshape)            (None, 11, 11, 50, 1 0           input_6[0][0]                    
__________________________________________________________________________________________________
conv3d_10 (Conv3D)              (None, 11, 11, 26, 1 26          reshape_10[0][0]                 
__________________________________________________________________________________________________
conv3d_11 (Conv3D)              (None, 11, 11, 1, 1) 27          conv3d_10[0][0]                  
____________________________________________________________________________________________

In [85]:
adam = Adam(lr=0.0001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [86]:
start = time.time()

In [ ]:
history = model.fit(x=[X_train_RNN, X_train_RES], y=y_train, epochs=100, batch_size=32, validation_data=([X_val_RNN, X_val_RES], y_val))

Train on 3950 samples, validate on 922 samples
Epoch 1/100
3950/3950 [==============================] - 92s 23ms/sample - loss: 6.6339 - accuracy: 0.2656 - val_loss: 6.4923 - val_accuracy: 0.2560
Epoch 2/100
3950/3950 [==============================] - 63s 16ms/sample - loss: 5.3627 - accuracy: 0.5208 - val_loss: 5.5057 - val_accuracy: 0.3720
Epoch 3/100
3950/3950 [==============================] - 71s 18ms/sample - loss: 4.5041 - accuracy: 0.6365 - val_loss: 4.6389 - val_accuracy: 0.5824
Epoch 4/100
3950/3950 [==============================] - 67s 17ms/sample - loss: 3.8560 - accuracy: 0.7149 - val_loss: 3.8051 - val_accuracy: 0.7299
Epoch 5/100
3950/3950 [==============================] - 71s 18ms/sample - loss: 3.2907 - accuracy: 0.8035 - val_loss: 3.1490 - val_accuracy: 0.8091
Epoch 6/100
3950/3950 [==============================] - 64s 16ms/sample - loss: 2.8247 - accuracy: 0.8661 - val_loss: 2.6303 - val_accuracy: 0.9447
Epoch 7/100
3950/3950 [==============================] - 64

In [ ]:
score = model.evaluate(x = [X_test_RNN, X_test_RES], y=y_test, batch_size=32)
Test_Loss =  score[0]
Test_accuracy = score[1]*100
print ("Test Accuracy : "+str(Test_accuracy))
print ("Test Loss : "+str(Test_Loss))

In [ ]:
end = time.time()
total = (end-start)/60
print ("Total time : "+str(total)+" minutes")

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc='center right')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc='center right')
plt.show()

In [ ]:
def plot_accuracy_map (X, y):
    Xshape = X
    X, y = createPatches(X, y, windowSize=windowSize, removeZeroLabels=False)
    X_RNN = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], 1)
    X_RES = X
    prediction = model.predict([X_RNN, X_RES])
    prediction = np.argmax(prediction, axis = 1)
    
    for i in range(0, len(y)):
        if (y[i] == 0):
            y[i] = 0
        elif (y[i] == 16):
            y[i] = 17
        else :
            y[i] = prediction[i]+1
            
    y = y.reshape((Xshape.shape[0], Xshape.shape[1]))
    predict_image = spectral.imshow(classes = y.astype(int),figsize =(5,5))

In [ ]:
#plot_accuracy_map(X_full, y_full)

In [ ]:
#predict_image = spectral.imshow(classes = y_full.astype(int),figsize =(5,5))